In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
import random

In [ ]:
from keras.applications import vgg16, inception_v3, resnet50, mobilenet
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.imagenet_utils import decode_predictions

In [ ]:
class line:
    
    def __init__(self, canvas, thickness=3, color=(0,0,0)):
        self.type = "line"
        self.p1 = (random.randint(0, canvas.shape[0]), random.randint(0, canvas.shape[1]))
        self.p2 = (random.randint(0, canvas.shape[0]), random.randint(0, canvas.shape[1]))
        self.thickness  = thickness
        self.color = color
        
    def draw_on(self, canvas):
        return cv2.line(canvas, self.p1, self.p2, self.color, self.thickness) 
        
class circle:
    
    def __init__(self, canvas, thickness=-1, color=(0,0,0), radius=-1):
        self.type = "circle"
        self.center = (random.randint(0, canvas.shape[0]), random.randint(0, canvas.shape[1]))
        self.radius = radius
        if self.radius==-1:
            self.radius = random.randint(10, canvas.shape[0]//2)
        self.thickness  = thickness
        self.color = color
   
    def draw_on(self, canvas):
        return cv2.circle(canvas, self.center, self.radius, self.color, self.thickness)
    
class arc:
    
    def __init__(self, canvas, thickness=-1, color=(0,0,0)):
        self.type = "circle"
        
        padding = 40
        
        self.center      = (random.randint(padding, canvas.shape[0]-padding), random.randint(padding, canvas.shape[0]-padding))
        self.radius1     = random.randint(20, 20+canvas.shape[0]-self.center[0])
        self.radius2     = random.randint(20, 20+canvas.shape[0]-self.center[1])
        self.angle       = random.randint(0, 360)
        self.start_angle = random.randint(0, 180)
        self.end_angle   = random.randint(self.start_angle+30, 360-30)
        self.thickness   = thickness
        self.color = color

    def draw_on(self, canvas):
        axes=(self.radius1, self.radius2)
        return cv2.ellipse(canvas, self.center, axes, self.angle, 
                           self.start_angle, self.end_angle, self.color, self.thickness)
    

class tableau:
    
    def __init__(self, shape, color=(255,255,255)):    
        self.canvas = np.ones((shape[0], shape[1], 3))*np.array(color)
        
    def display_with(self, entities):
        temp_canvas = np.copy(self.canvas)
        for entity in entities:
            temp_canvas = entity.draw_on(temp_canvas) 
        end_result = temp_canvas.astype(int)
        plt.imshow(end_result)
        return end_result

In [ ]:
#Load the VGG model
vgg_model = vgg16.VGG16(weights='imagenet')

#Load the ResNet50 model
resnet_model = resnet50.ResNet50(weights='imagenet')

#Load the MobileNet model
mobilenet_model = mobilenet.MobileNet(weights='imagenet')

In [ ]:
def predict(image, model=vgg_model):
    
    if model.name == "resnet50" or model.name == "mobilenet_1.00_224":
        # prepare the image for the VGG model
        processed_image = image.copy()

    elif model.name == "vgg16":
        
        # prepare the image for the VGG model
        processed_image = vgg16.preprocess_input(image.copy())
        #processed_image = image_batch.copy()

    # get the predicted probabilities for each class
    predictions = model.predict(image)
    # print predictions
    
    labels = decode_predictions(predictions, top=-1)
    
    return labels

def get_label_value(canvas, desired_label, model=vgg_model):

        labels = predict(np.expand_dims(canvas, axis=0))

        labels_array = np.array(labels[0])[:,1:]
        labels_dic   = dict(zip(labels_array[:,0], labels_array[:,1]))
        label        = labels_dic[desired_label]
        
        return label

In [ ]:
model = vgg_model

In [ ]:
desired_label = "rocking_chair"

j=0


while True:

    #color0 = (230,230,230)

    entities = []
    

    color0 = (random.randint(0,255), random.randint(0,255), random.randint(0,255))
    color1 = (random.randint(0,255), random.randint(0,255), random.randint(0,255))
    color2 = (random.randint(0,255), random.randint(0,255), random.randint(0,255))
    color3 = (random.randint(0,255), random.randint(0,255), random.randint(0,255))

    #color0 = (235, 156, 90)
    #color1 = (138, 114, 78)
    #color2 = (236, 217, 200)
    #color3 = (175, 74, 20)

    tableau1 = tableau([224,224], color=color0)

    #entities.append(arc(tableau1.canvas, thickness=100, color=np.array(color1)*.7))
    #entities.append(arc(tableau1.canvas, thickness=100, color=np.array(color1)*.7))

    entities.append(circle(tableau1.canvas, radius=30, thickness=-1, color=color3))

    entities.append(arc(tableau1.canvas, thickness=30, color=color1))
    entities.append(arc(tableau1.canvas, thickness=30, color=color1))
    entities.append(arc(tableau1.canvas, thickness=30, color=color1))


    labels = predict(np.expand_dims(tableau1.display_with(entities), 0), model)
    labels_array = np.array(labels[0])[:,1:]
    labels_dic   = dict(zip(labels_array[:,0], labels_array[:,1]))

    print("\nITERATION:{}".format(j))
    j+=1
    
    for i in range(5):
        print(i+1, list(labels_dic.keys())[i], "\t", labels_dic[list(labels_dic.keys())[i]])
        
    if list(labels_dic.keys())[0] == desired_label or labels_dic[list(labels_dic.keys())[0]]>.98:
        break

In [ ]:
# toilet_seat, nipple, 

In [ ]:
# rocking_chair, jigsaw_puzzle, iPod, street_sign, pinwheel